In [44]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Activation
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
import importlib

import utilis

In [45]:
# Check that you are running GPU's
utilis.GPU_checker()

You are runnning an instance with 4 GPU's


In [46]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7fb39e05c470>


# Model bi directional LSTM without hypotheses

In [47]:
sys.path.append('..')

In [48]:
from packages import generator

In [6]:
# generator config
W_HYP = False

In [51]:
%%time
#  generators
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator(data_dir='', subset_frac= 0.17, dataset='train_new', w_hyp=W_HYP)
validation_generator = generator.Keras_DataGenerator(data_dir='', subset_frac= 0.17, dataset='valid_new', w_hyp= W_HYP)

# of batches:  5888.0
# of batches reduced to:  1000.96
Generating examples from a set of 64061.44 examples 

# of batches:  1600.0
# of batches reduced to:  272.0
Generating examples from a set of 17408.0 examples 

CPU times: user 5.81 ms, sys: 374 µs, total: 6.19 ms
Wall time: 4.23 ms


In [52]:
#Constants
# ARE YOU LOADINNG MODE?
VOCAB_SIZE = 1254
INPUT_LENGTH = 1000
EMBEDDING_DIM = 512


# constnats
INPUT_LENGTH = 3000 if W_HYP else 1000
print(INPUT_LENGTH)


# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

1000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 512)         642048    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 1000, 512)         0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 41)                21033     
Total

In [53]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
#overwriting model
# model = loaded_model

In [61]:
%%time
#try and make it run until 9 am GMT+1
n_epochs = 10
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            verbose=1,
                            use_multiprocessing=True,
                            epochs=n_epochs)

Epoch 1/10
1000/1000 [==============================] - 953s 953ms/step - loss: 1.2855 - acc: 0.5553 - val_loss: 2.4990 - val_acc: 0.3148
Epoch 2/10
  13/1000 [..............................] - ETA: 4:01 - loss: 1.1749 - acc: 0.5721

Process ForkPoolWorker-56:
Process ForkPoolWorker-55:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, fun

KeyboardInterrupt: 

In [56]:
# FOR SAVING MODEL
model_json = model.to_json()
with open("model_seriously_overfitted.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_seriously_overfitted.h5")
print("Saved model to disk")

Saved model to disk


Process ForkPoolWorker-52:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/p

In [31]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "forjeff4"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [32]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS


In [57]:
test_generator = validation_generator = generator.Keras_DataGenerator(data_dir='', subset_frac= 0.125, dataset='test_new', w_hyp= W_HYP)

# of batches:  1920.0
# of batches reduced to:  240.0
Generating examples from a set of 15360.0 examples 



In [62]:
model.evaluate_generator(test_generator, verbose=1)

240/240 [==============================] - 139s 579ms/step


[2.5657591169079144, 0.31881510416666664]